# Deep Learning con Python

## Multilayer Perceptron (MLP)

En esta parte del taller construiremos y entrenaremos perceptrones multicapa (MLP) usando `tensorflow.keras` para resolver un problema de clasificación tabular relacionado con datos urbanos.

En concreto, **vamos a implementar una MLP para detectar si el nivel de polución en un área de una ciudad es alto o no en base a diferentes variables climátológicas, poblacionales y de uso del terreno urbano**.

---

In [ ]:
#Librería para poder visualizar imágenes
from IPython.display import Image, display

#

## 0) Flujo de trabajo

Para desarrollar desde 0 un algoritmo de Deep Learning capaz de procesar una serie de datos de entrada y proporcionar un servicio *inteligente* generalmente se sigue un flujo de trabajo definodo por 5 pasos concretos.

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_pipeline.jpg",width=800, height=300))

## 1) Librerías principales

Breve descripción de las librerías que usaremos:

- `pandas`, `numpy`: manejo y generación de datos.
- `matplotlib`, `seaborn`: visualización.
- `scikit-learn`: creación de dataset sintético, particionado, métricas y escalado.
- `tensorflow.keras`: definición, entrenamiento y evaluación de MLPs.

Ejecuta la siguiente celda para importar librerías.

In [ ]:
# Imports básicos (ejecutar en el notebook)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# reproducibilidad
RND = 42
np.random.seed(RND)


## 2) Lectura del dataset

En primer lugar vamos a leer el dataset con el que debemos trabajar. Puesto que dicho fichero se encuentra en formato CSV, vamos a hacer uso del método `read_csv` que aprendimos a usar en el notebook de `pandas`.

In [ ]:
# Leer el dataset desde CSV
df = pd.read_csv("https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/refs/heads/main/datos/city_mlp_dataset.csv")
if 'high_pollution' in df.columns:
    df['high_pollution'] = df['high_pollution'].astype(int)

print('Dataset cargado. Shape:', df.shape)

## 3) Análisis exploratorio de datos (sencillo)

Una vez que ya tenemos acceso a los datos con los que vamos a trabajar, en primer lugar es necesario familiarizarnos con los mismos e intentar comprenderlos bien. Esto se hace generalmente visualizando los datos e intentando computar determinádas estadísticas del dataset. Todo ello se suele denominar *Análisis Exploratorio de los Datos*.

Vamos a ver las primeras filas, descripción estadística y conteo de nulos.

In [ ]:
df.head()

In [ ]:
print('\nDescripción numérica:')
df.describe()

In [ ]:
# valores nulos
print('\nValores nulos por columna:')
df.isnull().sum()

Vamos a visualizar alguna fila en concreto que tenga valores NaN

In [ ]:
# Seleccionar filas que contienen al menos un NaN
filas_con_nan = df[df.isna().any(axis=1)]

# Mostrar solo 3 filas (si hay más, toma las primeras 3)
print("\nTres filas con al menos un NaN:")
display(filas_con_nan.head(3))

### Imputación de NaNs

Imputaremos los `NaN` de las columnas de atributos con la media. Nota de buenas prácticas: en un flujo real se debe calcular la imputación sólo en el conjunto de entrenamiento y aplicar esos valores al resto.

In [ ]:
# Imputar NaNs con la media (solo features)
feature_cols = df.columns.drop('high_pollution')
nans_before = df[feature_cols].isnull().sum().sum()
print(f'NaNs totales antes: {nans_before}')

In [ ]:
for col in feature_cols:
    if df[col].isnull().any():
        mean_val = df[col].mean()
        df.fillna({col:mean_val}, inplace=True)
nans_after = df[feature_cols].isnull().sum().sum()
print(f'NaNs totales después: {nans_after}')

In [ ]:
df.head()

### Visualizaciones rápidas: histogramas y matriz de correlación

Vamos a hacer una visualización sencilla de cómo están distribuidos los datos con los que vamos a trabajar mediante un gráfico de histogramas.

In [ ]:
# histogramas
plt.figure(figsize=(12,8))
df[feature_cols].hist(figsize=(12,8))
plt.suptitle('Histogramas de atributos (inspección rápida)')
plt.tight_layout()
plt.show()

Veamos ahora cuál es el coefficiente de correlación de Pearson entre cada par de variables.

In [ ]:
# correlación
plt.figure(figsize=(8,6))
cor = df.corr()
sns.heatmap(cor, annot=True, fmt='.2f')
plt.title('Matriz de correlación')
plt.show()

## 4) Particionado train/val/test y escalado

Cuando trabajamos con modelos de aprendizaje automático, es fundamental dividir los datos en conjuntos separados para asegurar que el modelo aprenda correctamente y podamos evaluar su desempeño de manera objetiva.

- Train (entrenamiento): es la parte más grande del dataset y se usa para que el modelo aprenda los patrones de los datos.

- Validation (validación): se utiliza durante el proceso de entrenamiento para ajustar los hiperparámetros y evitar el sobreajuste (overfitting). Sirve como una referencia intermedia para ver qué tan bien generaliza el modelo antes de probarlo con datos completamente nuevos.

- Test o *Hold out* (prueba): es un conjunto de datos separado, que no se ha usado ni en el entrenamiento ni en la validación. Se emplea al final para medir el rendimiento real del modelo sobre datos nunca vistos.

En resumen: train enseña al modelo, validation guía el ajuste, y test mide la capacidad de generalización.-

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/train_val_test_split.png",width=900, height=250))


Separamos en 60% train, 20% val, 20% test. Escalamos usando `StandardScaler` ajustado sólo con train.

In [ ]:
X = df.drop(columns=['high_pollution']).values
y = df['high_pollution'].values

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=RND)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=RND)

print('Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)




Cuando entrenamos una red neuronal MLP para trabajar con datos tabulares, es muy importante que las variables de entrada estén en escalas similares. Esto se debe a que:

- Diferencias de escala: En un conjunto de datos puede haber columnas con valores muy grandes (ej. ingresos anuales en miles de euros) y otras con valores muy pequeños (ej. edad en años). Si no normalizamos, la red tenderá a dar más importancia a las variables con números grandes, aunque no sean realmente más relevantes.

- Velocidad de entrenamiento: Los algoritmos de optimización (como Adam o SGD) funcionan mejor cuando los datos están centrados y con una escala similar. De lo contrario, el proceso de ajuste de pesos puede ser lento e ineficiente.

- Estabilidad numérica: Si los valores son demasiado grandes o demasiado pequeños, las funciones de activación (como sigmoid o tanh) pueden saturarse, lo que provoca que los gradientes se vuelvan casi nulos y la red aprenda muy poco.

Por eso se usa el `StandardScaler`, que transforma cada variable restando su media y dividiéndola por su desviación estándar. Como resultado, cada característica tiene media ≈ 0 y desviación estándar ≈ 1.

De esta manera, la MLP recibe todas las variables en una escala comparable, lo que mejora la eficiencia del entrenamiento y aumenta la precisión del modelo.

In [ ]:
# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled[:4]

In [ ]:
X_train[:4]

## 5) Nuestra primera Red Neuronal Artificial con Keras

### ¿Qué es Keras?

Keras es una **biblioteca de código abierto en Python** diseñada para facilitar la creación y el entrenamiento de modelos de **redes neuronales**.

### Características principales
- **Alto nivel de abstracción**: permite construir modelos de *deep learning* con pocas líneas de código.  
- **Integración con TensorFlow**: desde 2019 es la API oficial de alto nivel (`tf.keras`).  
- **Rapidez en prototipado**: útil para probar ideas nuevas rápidamente.  
- **Flexibilidad**: admite configuraciones avanzadas cuando se necesita.  
- **Ecosistema completo**: incluye módulos para redes densas, convolucionales, recurrentes, embeddings, dropout, etc.



### Conceptos básicos en entrenamiento de redes neuronales

- **Epoch**: una pasada completa por todos los datos de entrenamiento.  
- **Batch size**: número de muestras procesadas antes de actualizar los parámetros del modelo.  
- **Learning rate**: tamaño del paso que da el algoritmo al ajustar los pesos en cada actualización.  
- **Accuracy**: métrica que mide el porcentaje de predicciones correctas hechas por el modelo.  


### Red Neuronal Multilayer Preceptron (MLP)

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_mlp_arch.jpg",width=1000, height=500))

Creamos el código que permita definir nuestra MLP. En este caso, las capas serán:

- 1 de entrada (`Input`)
- 1 oculta (`Dense`)
- 1 de salida con una única neurona (`Dense`)

Es importante destacar porqué usamos la funcion sigmoide (`sigmoid`) en la capa de salida. En problemas de clasificación binaria, la función *sigmoide* se utiliza en la capa de salida por varios motivos:

- Convierte la salida del modelo en un valor entre 0 y 1.  
- Ese valor puede interpretarse como la *probabilidad* de pertenecer a la clase positiva (clase 1).  
- Permite tomar decisiones aplicando un umbral (por defecto, 0.5):  
  - Si la salida ≥ 0.5 → se predice la clase 1.  
  - Si la salida < 0.5 → se predice la clase 0.  


In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_funcion_sigmoide.png",width=400, height=400))

In [ ]:
def construir_mlp_simple(input_dim, lr=1e-3):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = construir_mlp_simple(X_train_scaled.shape[1], lr=1e-3)
model.summary()

In [ ]:
# Entrenamiento
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_val_scaled, y_val),
                    epochs=30,
                    batch_size=32,
                    verbose=2)

### Learning curves en el entrenamiento de una red neuronal

Las curvas de aprendizaje son gráficos que muestran cómo evoluciona el rendimiento de un modelo a lo largo del entrenamiento.  
Normalmente se representan dos curvas:  
- *Pérdida (loss)* en entrenamiento y validación.  
- *Métrica de calidad* (en nuestro caso, accuracy) en entrenamiento y validación.  

#### ¿Por qué son importantes?
Revisarlas permite:
- Detectar *overfitting*: cuando el rendimiento en entrenamiento mejora pero en validación empeora.  
- Detectar *underfitting*: cuando el modelo no logra un buen rendimiento ni en entrenamiento ni en validación.  
- Ajustar *hiperparámetros* como número de *epochs*, *learning rate* o complejidad de la red.  
- Verificar si el modelo *sigue aprendiendo* o si ya se ha estancado.  



In [ ]:
# Curvas
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Evaluación en test
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test loss: {test_loss:.4f}  |  Test accuracy: {test_acc:.4f}")

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_accuracy_recall.png",width=350, height=400))

In [ ]:
# Reporte
y_pred_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)
print('\nClassification report:')
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
print('\nConfusion matrix:')
print(confusion_matrix(y_test, y_pred))

## 6) Variantes y EarlyStopping

Vamos a probar algunas variantes de MLP añadiendole algunas capas ocultas más y cambiando su número de neuronas.

In [ ]:
def construir_variante_mlp(input_dim, hidden_units, lr=1e-3):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    for units in hidden_units:
        model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

También vamos a usar `early stopping`, una técnica de regularización que consiste en detener el entrenamiento de una red neuronal antes de que termine el número máximo de *epochs* programadas.  

Durante el entrenamiento se monitoriza una métrica en el conjunto de validación.  Si esa métrica deja de mejorar después de cierto número de *epochs consecutivas* (llamado *patience*), el entrenamiento se interrumpe automáticamente.  

De esta forma, se evita el *overfitting*, ya que el modelo deja de entrenar justo antes de empezar a memorizar demasiado los datos de entrenamiento, ahorra tiempo y recursos de cómputo y suele producir un modelo con mejor capacidad de generalización en datos nuevos.  



In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_early_stopping.png",width=400, height=380))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

variants = {
    'A_64_32': [64,32],
    'B_128_64_32': [128,64,32]
}

results = {}

for name, hidden in variants.items():
    print('\n=== Entrenando variante', name, 'con capas', hidden, '===')
    model_var = construir_variante_mlp(X_train_scaled.shape[1], hidden, lr=1e-3)
    early = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
    hist = model_var.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=100,
                         batch_size=32,
                         callbacks=[early],
                         verbose=2)
    loss, acc = model_var.evaluate(X_test_scaled, y_test, verbose=0)
    y_p = (model_var.predict(X_test_scaled).ravel() >= 0.5).astype(int)
    rep = classification_report(y_test, y_p)
    results[name] = {
        'model': model_var,
        'history': hist,
        'test_loss': loss,
        'test_acc': acc,
        'report': rep
    }
    print(f"Variante {name} -> Test acc: {acc:.4f}")

In [ ]:
# Resumen

for name, info in results.items():
    print(name)
    print(info['report'])
    print('---')

¡Eso es todo amigos!
